In [95]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [96]:
import numpy as np
import pandas as pd
import operator
import os
import glob


In [97]:
#importing required libraries
import nltk
#downloading stopwords,wordnet from nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [98]:
title = "/content/drive/MyDrive/kkk/storiess"
#reading the files path
paths = []
for (dirpath, dirnames, filenames) in os.walk(str(title)):
    for i in filenames:
        paths.append(str(dirpath)+str("/")+i)

In [99]:
print(len(paths))
print(paths[0])
print(paths[1])

467
/content/drive/MyDrive/kkk/storiess/quarter.c3
/content/drive/MyDrive/kkk/storiess/bookem2


QUES1_PARTA-
preprocessing

In [100]:
#preprocessing part
def remove_header(data):
    try:
        ind = data.index('\n\n')
        data = data[ind:]
    except:
        print("No Header")
    return data
#converting lower case
def convert_lower_case(data):
    return np.char.lower(data)
#removing stop words
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

#removing punctuation
def remove_punctuation(data):
    symbols = "/:;!\"#+-.&()*^_`<=>?@[\]$%{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', ' ')
    data = np.char.replace(data, "'", "")
    return data

#removing single character
def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)   

#converting numbers into words
def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data     

#stemmitization
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return np.char.strip(new_text) 
#calling all preprocessing functions
def preprocess(data): 
         
    data = convert_lower_case(data)
    data = convert_numbers(data)
    data = remove_stop_words(data)
    data = remove_punctuation(data) 
    data = remove_single_characters(data)

    return data       

Q1_b positional indexes

In [101]:
doc = 0
postings = pd.DataFrame()
frequency = pd.DataFrame()
fileIdName = []
files_tokens= []

for path in paths:
    file = open(path, 'r', encoding= 'unicode_escape')
    text = " ".join(file.read().split()) 
    file.close()
    preprocessed_text = preprocess(text)  
    
    # preprocessing of input files is done.
    print(doc)
    #tokenization is done here
    tokens = word_tokenize(str(preprocessed_text))
    files_tokens.append(tokens)
    pos = 0
    #creating postings
    for token in tokens:
        if token in postings:
            p = postings[token][0]            

            k = [a[0] for a in p]
            if doc in k:
                for a in p:
                    if a[0] == doc:
                        a[1].add(pos)
            else:
                p.append([doc,{pos}])
                frequency[token][0] += 1
        else:
            postings.insert(value=[[[doc, {pos}]]], loc=0, column=token)
            frequency.insert(value=[1], loc=0, column=token)

        pos += 1


    filename = os.path.basename(path)
    fileIdName.append([filename])
    doc += 1

#filedName stores names of file
fileIdName = pd.DataFrame(fileIdName)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [102]:
print(files_tokens[0])

['secrets', 'start', 'republic', 'stops', 'norman', 'thomas', 'quarantine', 'moses', 'domingo', 'welcome', 'good', 'old', 'land', 'gangs', 'land', 'ignored', 'deprived', 'land', 'dead', 'soil', 'nuclear', 'weapons', 'illiterate', 'population', 'it', 'feels', 'great', 'american', 'to', 'go', 'outsideand', 'walk', 'upon', 'dead', 'trampled', 'stretch', 'arms', 'inhale', 'dry', 'air', 'walk', 'field', 'pretend', 'trees', 'does', 'nt', 'feel', 'great', 'manners', 'hello', 'name', 'joe', 'simmons', 'drummer', 'please', 'enter', 'joe', 'world', 'two', 'zero', 'five', 'three', 'where', 'nightmare', 'reality', 'month', 'since', 'suicide', 'attempt', 'still', 'shudder', 'thought', 'trying', 'blow', 'brains', 'day', 'went', 'cemetery', 'visited', 'graves', 'family', 'girlfriend', 'stood', 'would', 'occasionally', 'look', 'sky', 'rather', 'atmospheric', 'shield', 'simulation', 'sky', 'and', 'prayed', 'would', 'actually', 'rain', 'set', 'perfect', 'mood', 'give', 'everyone', 'earth', 'big', 'break

In [103]:
postings

,disengaging,pentration,royalship,phalanxes,invades,interrogating,subduing,navterm,bov,wingmen,latching,shockwace,nalsi,dissatisfaction,subcommodore,assassinated,subcommodores,grosne,couriers,coo,tuh,nioxyia,trainees,assitants,rybosi,forensics,orinaka,underinspector,refregi,triumphing,viewports,yours,refreji,wasalia,opertions,underchief,honawa,dina,rufreji,frezar,...,trees,pretend,field,air,dry,inhale,arms,stretch,trampled,upon,walk,outsideand,go,to,american,great,feels,it,population,illiterate,weapons,nuclear,soil,dead,deprived,ignored,gangs,land,old,good,welcome,domingo,moses,quarantine,thomas,norman,stops,republic,start,secrets
0,"[[466, {1773}]]","[[466, {1673}]]","[[466, {1668, 1706, 1719, 1723, 1755}]]","[[466, {1616}]]","[[466, {1510}]]","[[466, {1475}]]","[[466, {1467}]]","[[466, {1361}]]","[[466, {1339, 1332}]]","[[466, {1298}]]","[[466, {1277}]]","[[466, {1176}]]","[[466, {1144, 1098, 1164}]]","[[466, {862}]]","[[466, {852}]]","[[466, {851}]]","[[466, {843}]]","[[466, {1452, 1587, 1524, 821, 1716, 1464, 829}]]","[[466, {800}]]","[[466, {790, 791}]]","[[466, {789}]]","[[466, {731, 739}]]","[[466, {699}]]","[[466, {695}]]","[[466, {686}]]","[[466, {688, 671}]]","[[466, {673, 705, 741, 710, 719, 656, 752, 691...","[[466, {896, 1473, 997, 1095, 714, 1648, 1457,...","[[466, {620}]]","[[466, {550}]]","[[466, {499}]]","[[466, {438}]]","[[466, {435, 469}]]","[[466, {424}]]","[[466, {371}]]","[[466, {273}]]","[[466, {272, 512, 495, 479}]]","[[466, {492, 269, 478, 511}]]","[[466, {231}]]","[[466, {230}]]",...,"[[0, {888, 43, 896}], [24, {104, 93}], [26, {6...","[[0, {42}], [14, {680}], [39, {84}], [46, {559...","[[0, {41}], [13, {1416, 1580, 1495}], [18, {73...","[[0, {39}], [3, {335}], [8, {6}], [13, {6467, ...","[[0, {38}], [3, {79, 15}], [13, {2395}], [22, ...","[[0, {37}], [190, {10100}], [327, {5546}], [35...","[[0, {36, 270}], [3, {370, 230}], [4, {192}], ...","[[0, {627, 35}], [29, {1393, 5529}], [31, {146...","[[0, {1441, 34, 429}], [62, {1526}], [112, {41...","[[0, {32, 1442, 430, 863}], [1, {776}], [4, {5...","[[0, {40, 31}], [1, {303}], [3, {88}], [6, {36...","[[0, {30}]]","[[0, {29, 1326, 575}], [1, {82, 388, 358, 719}...","[[0, {28}], [37, {556, 501}], [62, {2228, 1476...","[[0, {27}], [23, {393}], [33, {430}], [46, {18...","[[0, {1443, 1068, 431, 47, 26}], [1, {914}], [...","[[0, {25}], [46, {44525, 16375}], [62, {2053, ...","[[0, {24, 675, 716, 382}], [13, {5186, 5635, 5...","[[0, {1343, 23}], [13, {4220, 4693}], [23, {40...","[[0, {22}], [394, {30065}], [429, {3986}]]","[[0, {21}], [18, {294}], [29, {8929, 9123, 848...","[[0, {20}], [33, {640}], [47, {76}], [53, {72}...","[[0, {19}], [13, {2991}], [31, {998}], [46, {3...","[[0, {33, 18}], [11, {253}], [13, {848, 4790}]...","[[0, {16}], [112, {2762}], [163, {16168, 8195,...","[[0, {15}], [13, {642, 4356}], [33, {382}], [4...","[[0, {13}], [11, {570}], [29, {6550}], [56, {2...","[[0, {17, 12, 14}], [20, {360, 830, 743}], [23...","[[0, {11, 660, 375}], [1, {433, 935}], [6, {28...","[[0, {1377, 10, 1179}], [1, {732}], [3, {33}],...","[[0, {9}], [1, {35}], [3, {0}], [20, {840}], [...","[[0, {8}]]","[[0, {7}], [155, {12200, 12195, 12235}], [264,...","[[0, {388, 902, 6, 690, 759}], [321, {431}]]","[[0, {5}], [58, {2, 1925}], [61, {2, 514}], [9...","[[0, {4}], [123, {2298, 2188}], [245, {2252}],...","[[0, {3}], [22, {355}], [46, {8681}], [51, {94...","[[0, {2}], [133, {6699}], [190, {3885}], [195,...","[[0, {1536, 1}], [13, {769, 774}], [14, {1016,...","[[0, {0}], [34, {352, 476}], [43, {199}], [46,..."


In [104]:
print(fileIdName)

                0
0      quarter.c3
1         bookem2
2      quarter.c9
3     quarter.c12
4      quarter.c1
..            ...
462     sre10.txt
463   sre_sei.txt
464      srex.txt
465     sre04.txt
466  sre_feqh.txt

[467 rows x 1 columns]


In [105]:
frequency

,disengaging,pentration,royalship,phalanxes,invades,interrogating,subduing,navterm,bov,wingmen,latching,shockwace,nalsi,dissatisfaction,subcommodore,assassinated,subcommodores,grosne,couriers,coo,tuh,nioxyia,trainees,assitants,rybosi,forensics,orinaka,underinspector,refregi,triumphing,viewports,yours,refreji,wasalia,opertions,underchief,honawa,dina,rufreji,frezar,...,trees,pretend,field,air,dry,inhale,arms,stretch,trampled,upon,walk,outsideand,go,to,american,great,feels,it,population,illiterate,weapons,nuclear,soil,dead,deprived,ignored,gangs,land,old,good,welcome,domingo,moses,quarantine,thomas,norman,stops,republic,start,secrets
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,101,24,100,196,79,4,145,36,9,246,142,1,314,78,55,238,20,63,28,3,77,30,32,187,14,46,10,108,281,296,80,1,3,2,18,6,22,8,138,33


In [106]:
#save the posting and fileIdname that can we build futher with run above code.
postings.to_pickle("posting")
frequency.to_pickle("frequency")
fileIdName.to_pickle("fileIdName")

In [107]:
#read the posting and fileIdName 
postings = pd.read_pickle("posting")
frequency=pd.read_pickle("frequency")
fileIdName = pd.read_pickle("fileIdName")

q1_3-Query


In [108]:
#########for searching query
def find_postings_of_words(word):
  #preprocessing words
    preprocessed_word = str(preprocess(word))
    print(preprocessed_word)
    print("Frequency of the word:",frequency[preprocessed_word][0])
    print("Postings List are:",postings[preprocessed_word][0])

In [109]:
#getting positional indexes
def find_postions(postingvalues, documnet):
    for postingvalue in postingvalues:
        if postingvalue[0] == documnet:
            return postingvalue[1]
    return {}

In [110]:
#generating words with thier positions
def gen_init_matchingSets(word):
    initial = []
    wordPostings = postings[word][0]
    for wordPosting in wordPostings:
        for positions in wordPosting[1]:
            initial.append((wordPosting[0], positions))
    return initial

In [111]:
#matching postional index
def get_match_of_positional_index(initial, r):
    matched_docs = []
    for p in initial:
        pos = p[1]
        doc = p[0]
       

        count = 0

        for k in r:
            pos = pos+1
            k_pos = postings[k][0]
            document_list = [z[0] for z in k_pos]
            if doc in document_list:
                doc_positions = find_postions(k_pos, doc)
                if pos in doc_positions:
                    count += 1
                else:
                    count += 1
                    break

            if count == len(r):
                matched_docs.append(p[0])
    return set(matched_docs)


In [112]:
#function for query testing
def queryRun(query):
    #preprocessing the query
    processedQuery = preprocess(query)
    print( processedQuery )
    #tokenizing the query
    tokens = word_tokenize(str(processedQuery))
    print(tokens)
    
    if len(tokens)==1:
        print("There is  1 word only ")
        print("Number of Document Mathced are", [a[0] for a in postings[query][0]])
        return [a[0] for a in postings[query][0]]
    
    ini_word = tokens[0]
    print("iniword",ini_word)
    ini_matches = gen_init_matchingSets(ini_word)
    print("inimatches",ini_matches)
    tokens.pop(0)
    print(tokens)
    #getting number of documnets matched
    total_matched_docs = get_match_of_positional_index(ini_matches, tokens)
    print("Number of Document Mathced are:", total_matched_docs)
    return total_matched_docs

In [113]:
def print_file(file):
    output_file = open(paths[file], 'r', encoding='cp1250')
    output_text = output_file.read()
    print(output_text)

In [114]:
find_postings_of_words("good")

good
Frequency of the word: 296
Postings List are: [[0, {1377, 10, 1179}], [1, {732}], [3, {33}], [5, {177, 211, 100}], [13, {136, 2952, 1548, 4108, 534, 6806, 162, 5154, 548, 1574, 4647, 4905, 4652, 5292, 1464, 1873, 2385, 5467, 3428, 3045, 6629, 1653, 506}], [14, {393, 701, 1297, 1350}], [16, {211}], [18, {28, 21}], [20, {756}], [23, {301}], [29, {192, 6592, 2435, 10116, 2596, 2761, 4106, 12977, 4050, 2387, 6451, 13332, 2614, 3320, 12346, 10652, 5501, 6463}], [31, {99}], [32, {454}], [33, {412}], [34, {219}], [35, {241}], [36, {578, 667, 573}], [37, {462}], [38, {105}], [39, {800, 390, 456, 969, 490, 652, 429, 975, 720, 471}], [44, {105}], [45, {677}], [46, {2569, 31761, 25107, 34838, 58905, 51225, 3100, 24612, 58919, 15400, 21544, 3114, 49194, 53809, 44087, 45134, 55887, 41552, 8275, 30302, 35424, 26210, 8806, 10855, 22121, 6255, 10868, 22647, 42105, 35962, 2176, 51332, 6279, 10888, 24711, 27786, 32903, 26764, 29324, 32398, 51350, 47268, 25765, 29864, 26284, 31916, 47283, 21176, 350

In [115]:
#inputting the query
query = input('Enter the query  : ')

Enter the query  : good day


In [116]:
list1 = queryRun(query)
number_of_document_match=len(list1)

number_of_document_match
name_list = []
for i in list1:
  name_list.append(fileIdName.iloc[i][0])
#printing output(files names and number of documnet matches)
print(name_list)
print("total document matches",number_of_document_match)

good day
['good', 'day']
iniword good
inimatches [(0, 1377), (0, 10), (0, 1179), (1, 732), (3, 33), (5, 177), (5, 211), (5, 100), (13, 136), (13, 2952), (13, 1548), (13, 4108), (13, 534), (13, 6806), (13, 162), (13, 5154), (13, 548), (13, 1574), (13, 4647), (13, 4905), (13, 4652), (13, 5292), (13, 1464), (13, 1873), (13, 2385), (13, 5467), (13, 3428), (13, 3045), (13, 6629), (13, 1653), (13, 506), (14, 393), (14, 701), (14, 1297), (14, 1350), (16, 211), (18, 28), (18, 21), (20, 756), (23, 301), (29, 192), (29, 6592), (29, 2435), (29, 10116), (29, 2596), (29, 2761), (29, 4106), (29, 12977), (29, 4050), (29, 2387), (29, 6451), (29, 13332), (29, 2614), (29, 3320), (29, 12346), (29, 10652), (29, 5501), (29, 6463), (31, 99), (32, 454), (33, 412), (34, 219), (35, 241), (36, 578), (36, 667), (36, 573), (37, 462), (38, 105), (39, 800), (39, 390), (39, 456), (39, 969), (39, 490), (39, 652), (39, 429), (39, 975), (39, 720), (39, 471), (44, 105), (45, 677), (46, 2569), (46, 31761), (46, 25107), (

**Question2 Scoring and Term-Weighting**


**2.1-----Jaccard Coefficient**

In [117]:
#Union of document token and query token
def union_doc_query(doc,query):
    union_list = list(set(doc) | set(query))
    return union_list

#intersection of document token and query token
def intersection_doc_query(doc,query):
    intersection_list = list(set(doc) & set(query))
    return intersection_list

In [118]:
#function for finding Jaccard Coefficient
def Jaccard_Coefficient(query_list):
    JaccardCoeff_dict = {}
    document_id = 0
    #files_tokens is storing the tokens of each doc in the form of list of list item refers to one doc so it's a document token
    for item in files_tokens: 
        intersection = intersection_doc_query(item,query_list)
        union = union_doc_query(item,query_list)

        #Jaccard Coefficient = Intersection of (doc,query) / Union of (doc,query)
        JaccardCoeff_dict[document_id] = len(intersection)/len(union)
    
        document_id = document_id + 1  #counting the number of documents for fetching top 5 docs

        #Sorting the jaccard coefficient in decreasing order 
        result = dict(sorted(JaccardCoeff_dict.items(),key=operator.itemgetter(1),reverse=True))
            
    return result

In [119]:
#query token is calculated
def query_tokens(query):
    #print("hiii")
    processed_query = preprocess(query)        #preprocessing of query
    #print(processed_query)
    #print("yyyyy")
    query_token = word_tokenize(str(processed_query))
    #print(query_token)
    return query_token

In [120]:
 #fetching top 5 relevant documents based on the value of the Jaccard coefficient
def jaccard_coeff_func(query):
    query_afterPreprocess =query_tokens(query)  #tokeinising the query
    print('input query tokens are',query_afterPreprocess)
    after_jc = Jaccard_Coefficient(query_afterPreprocess)
    print('jaccard coefficient of docs',after_jc)
    count = -1
    jc_top5_list = []
    for item in after_jc.keys():
        count += 1
        #print(count)
        if count == 5:
            print('top 5 relevant documents based on the value of the Jaccard coefficient are:')
            print(jc_top5_list)
            for i in jc_top5_list:
                print(fileIdName.iloc[i][0])
            return
        jc_top5_list.append(item)

In [121]:
jc_query = input('Enter the query for fetching top 5 docs based on jaccard coefficient')
jaccard_coeff_func(jc_query)

Enter the query for fetching top 5 docs based on jaccard coefficientgood day
input query tokens are ['good', 'day']
jaccard coefficient of docs {15: 0.02, 82: 0.017543859649122806, 415: 0.016129032258064516, 353: 0.015625, 147: 0.015384615384615385, 5: 0.015267175572519083, 150: 0.015151515151515152, 92: 0.014388489208633094, 98: 0.013888888888888888, 312: 0.013888888888888888, 416: 0.013793103448275862, 211: 0.013605442176870748, 77: 0.012987012987012988, 265: 0.012658227848101266, 64: 0.011904761904761904, 442: 0.011494252873563218, 269: 0.01098901098901099, 277: 0.010526315789473684, 386: 0.01020408163265306, 418: 0.01020408163265306, 75: 0.010101010101010102, 235: 0.00980392156862745, 2: 0.009708737864077669, 214: 0.009345794392523364, 100: 0.008849557522123894, 261: 0.00847457627118644, 68: 0.008403361344537815, 139: 0.008264462809917356, 183: 0.008264462809917356, 238: 0.008264462809917356, 307: 0.008264462809917356, 393: 0.008130081300813009, 376: 0.00796812749003984, 49: 0.0079

**2.2----TFIDF**


***Raw Count Variant***

In [122]:
#5 different varaints for calculating term frequency
#1-Raw Count-f(t,d)-----frequency of each term in doc

Raw_Count_Frequency = []   #list of list for storing raw count term frequency
for item in files_tokens:
    dict_raw_tf ={}      #dictionary for storing raw count term frequency
    for w in item:
        if dict_raw_tf.get(w) == None:  
            dict_raw_tf[w] = 1       #if it's a new word then count=1
        else:
            count = dict_raw_tf.get(w)
            count += 1               #else increase the count by 1
            dict_raw_tf[w] = count
    Raw_Count_Frequency.append(dict_raw_tf)      #dictionary storing raw count frequency of each term in doc

In [123]:
Raw_Count_Frequency[0]  #for doc 1

{'ability': 1,
 'absorbed': 1,
 'accompanying': 1,
 'act': 1,
 'actually': 3,
 'add': 1,
 'adequately': 1,
 'administer': 1,
 'age': 1,
 'aged': 3,
 'ages': 1,
 'air': 1,
 'alberta': 1,
 'all': 1,
 'almost': 2,
 'alone': 2,
 'along': 2,
 'altered': 1,
 'although': 1,
 'always': 1,
 'american': 1,
 'among': 1,
 'ancestors': 1,
 'and': 3,
 'angeles': 1,
 'animals': 1,
 'ankle': 1,
 'another': 3,
 'answer': 2,
 'antartica': 1,
 'apartment': 1,
 'apparent': 1,
 'appeared': 1,
 'area': 1,
 'argument': 1,
 'arm': 1,
 'arms': 2,
 'around': 6,
 'arranged': 1,
 'array': 1,
 'arrested': 1,
 'ask': 1,
 'atmospheric': 4,
 'attempt': 2,
 'avoid': 1,
 'awakens': 1,
 'away': 1,
 'baby': 1,
 'babylon': 1,
 'back': 4,
 'bag': 1,
 'band': 1,
 'bank': 1,
 'barriers': 1,
 'base': 1,
 'based': 1,
 'bastard': 1,
 'beautiful': 1,
 'becoming': 1,
 'began': 6,
 'behind': 1,
 'beings': 1,
 'believed': 1,
 'belonged': 1,
 'berk': 1,
 'berkin': 7,
 'big': 3,
 'birth': 1,
 'birthdate': 1,
 'birthday': 1,
 'bitchin

***Term Frequency Variant***

In [124]:
#2-f(t,d)/∑f(t`,d)---frequncy of term divide by all no. of words in doc
Term_Frequency = []     #list of list for stroing log normalization term frequency
for item in Raw_Count_Frequency:
    dict_tf = {}        #dictionary for stroing term frequency
    allkeys = item.keys()
    sum_tf = sum(item.values())     #counting no. of tokens in each document
    for w in allkeys:
        freq = item.get(w)
        t_freq =(freq/sum_tf)   #divide each term frquency by total no. of tokens in doc
        dict_tf[w] = t_freq
    Term_Frequency.append(dict_tf)

In [125]:
Term_Frequency[0]

{'ability': 0.0006464124111182935,
 'absorbed': 0.0006464124111182935,
 'accompanying': 0.0006464124111182935,
 'act': 0.0006464124111182935,
 'actually': 0.0019392372333548805,
 'add': 0.0006464124111182935,
 'adequately': 0.0006464124111182935,
 'administer': 0.0006464124111182935,
 'age': 0.0006464124111182935,
 'aged': 0.0019392372333548805,
 'ages': 0.0006464124111182935,
 'air': 0.0006464124111182935,
 'alberta': 0.0006464124111182935,
 'all': 0.0006464124111182935,
 'almost': 0.001292824822236587,
 'alone': 0.001292824822236587,
 'along': 0.001292824822236587,
 'altered': 0.0006464124111182935,
 'although': 0.0006464124111182935,
 'always': 0.0006464124111182935,
 'american': 0.0006464124111182935,
 'among': 0.0006464124111182935,
 'ancestors': 0.0006464124111182935,
 'and': 0.0019392372333548805,
 'angeles': 0.0006464124111182935,
 'animals': 0.0006464124111182935,
 'ankle': 0.0006464124111182935,
 'another': 0.0019392372333548805,
 'answer': 0.001292824822236587,
 'antartica':

***Log Normalization Varaiant***

In [126]:
#3- Log Normalisation = log(1 +f(t,d)))
Log_Normalisation = []       #list of list for stroing double normalization term frequency of each doc
for item in Raw_Count_Frequency:
    dict_log_tf = {}         #dictionary for stroing log normalization term frequency
    allkeys = item.keys()
    for w in allkeys:
        freq = item.get(w)
        #print(freq)
        log_freq = np.log(1+freq)     # log(1 +f(t,d)))
        dict_log_tf[w] = log_freq    
    Log_Normalisation.append(dict_log_tf)

In [127]:
Log_Normalisation[0]

{'ability': 0.6931471805599453,
 'absorbed': 0.6931471805599453,
 'accompanying': 0.6931471805599453,
 'act': 0.6931471805599453,
 'actually': 1.3862943611198906,
 'add': 0.6931471805599453,
 'adequately': 0.6931471805599453,
 'administer': 0.6931471805599453,
 'age': 0.6931471805599453,
 'aged': 1.3862943611198906,
 'ages': 0.6931471805599453,
 'air': 0.6931471805599453,
 'alberta': 0.6931471805599453,
 'all': 0.6931471805599453,
 'almost': 1.0986122886681098,
 'alone': 1.0986122886681098,
 'along': 1.0986122886681098,
 'altered': 0.6931471805599453,
 'although': 0.6931471805599453,
 'always': 0.6931471805599453,
 'american': 0.6931471805599453,
 'among': 0.6931471805599453,
 'ancestors': 0.6931471805599453,
 'and': 1.3862943611198906,
 'angeles': 0.6931471805599453,
 'animals': 0.6931471805599453,
 'ankle': 0.6931471805599453,
 'another': 1.3862943611198906,
 'answer': 1.0986122886681098,
 'antartica': 0.6931471805599453,
 'apartment': 0.6931471805599453,
 'apparent': 0.6931471805599

***Double Normalisation Variant***

In [128]:
#4---Double Normalisation Frequency = 0.5 + ((0.5*tf)/max(tf))
Double_Normalisation = []  #list of list for storing double normalization term frequency for each doc
for item in Raw_Count_Frequency:
    dict_dn_tf = {}             #dictionary for storing double normalization term frequency
    allkeys = item.keys()
    max_tf = max(item.values())
    for w in allkeys:
        freq = item.get(w)
        aug_freq = 0.5 + (0.5*(freq/max_tf))    #Double Normalisation Frequency = 0.5 + ((0.5*tf)/max(tf))
        dict_dn_tf[w] = aug_freq
    Double_Normalisation.append(dict_dn_tf)

In [129]:
Double_Normalisation[0]

{'ability': 0.5172413793103449,
 'absorbed': 0.5172413793103449,
 'accompanying': 0.5172413793103449,
 'act': 0.5172413793103449,
 'actually': 0.5517241379310345,
 'add': 0.5172413793103449,
 'adequately': 0.5172413793103449,
 'administer': 0.5172413793103449,
 'age': 0.5172413793103449,
 'aged': 0.5517241379310345,
 'ages': 0.5172413793103449,
 'air': 0.5172413793103449,
 'alberta': 0.5172413793103449,
 'all': 0.5172413793103449,
 'almost': 0.5344827586206896,
 'alone': 0.5344827586206896,
 'along': 0.5344827586206896,
 'altered': 0.5172413793103449,
 'although': 0.5172413793103449,
 'always': 0.5172413793103449,
 'american': 0.5172413793103449,
 'among': 0.5172413793103449,
 'ancestors': 0.5172413793103449,
 'and': 0.5517241379310345,
 'angeles': 0.5172413793103449,
 'animals': 0.5172413793103449,
 'ankle': 0.5172413793103449,
 'another': 0.5517241379310345,
 'answer': 0.5344827586206896,
 'antartica': 0.5172413793103449,
 'apartment': 0.5172413793103449,
 'apparent': 0.5172413793103

***Binary Variant***

In [130]:
#5------Binary 0,1
Binary_Term_Frequency = []   #list of list for storing binary term frequency
for item in Raw_Count_Frequency:
    dict_bin_tf = {}         #dictionary for storing binary term frequency
    allkeys = item.keys()
    for w in allkeys:
        freq = item.get(w)
        if freq > 0:          
            bool_freq = 1   #if frequency >0 then it's 1
        else:
            bool_freq = 0       #else 0
        dict_bin_tf[w] = bool_freq
    Binary_Term_Frequency.append(dict_bin_tf)

In [131]:
Binary_Term_Frequency[0]

{'ability': 1,
 'absorbed': 1,
 'accompanying': 1,
 'act': 1,
 'actually': 1,
 'add': 1,
 'adequately': 1,
 'administer': 1,
 'age': 1,
 'aged': 1,
 'ages': 1,
 'air': 1,
 'alberta': 1,
 'all': 1,
 'almost': 1,
 'alone': 1,
 'along': 1,
 'altered': 1,
 'although': 1,
 'always': 1,
 'american': 1,
 'among': 1,
 'ancestors': 1,
 'and': 1,
 'angeles': 1,
 'animals': 1,
 'ankle': 1,
 'another': 1,
 'answer': 1,
 'antartica': 1,
 'apartment': 1,
 'apparent': 1,
 'appeared': 1,
 'area': 1,
 'argument': 1,
 'arm': 1,
 'arms': 1,
 'around': 1,
 'arranged': 1,
 'array': 1,
 'arrested': 1,
 'ask': 1,
 'atmospheric': 1,
 'attempt': 1,
 'avoid': 1,
 'awakens': 1,
 'away': 1,
 'baby': 1,
 'babylon': 1,
 'back': 1,
 'bag': 1,
 'band': 1,
 'bank': 1,
 'barriers': 1,
 'base': 1,
 'based': 1,
 'bastard': 1,
 'beautiful': 1,
 'becoming': 1,
 'began': 1,
 'behind': 1,
 'beings': 1,
 'believed': 1,
 'belonged': 1,
 'berk': 1,
 'berkin': 1,
 'big': 1,
 'birth': 1,
 'birthdate': 1,
 'birthday': 1,
 'bitchin

***Document Frequency***

In [132]:
#Document Frequency --no. of docs in which term t present
doc_frequency = {}    #stroing doc frequency of each term
for item in files_tokens:
    terms = list(set(item))
    for w in terms:
        if doc_frequency.get(w) == None:
            doc_frequency[w] = 1   #if word coming first time then count = 1
        else:
            count = doc_frequency.get(w)
            count += 1              #else count frequency by incrementing in each doc
            doc_frequency[w] = count

In [133]:
doc_frequency

{'found': 274,
 'atmospheric': 5,
 'offer': 72,
 'door': 243,
 'years': 233,
 'isaak': 1,
 'liquid': 40,
 'physical': 55,
 'outside': 156,
 'toward': 125,
 'white': 182,
 'possibly': 86,
 'zero': 259,
 'st': 48,
 'sworn': 24,
 'cried': 115,
 'mouth': 163,
 'match': 60,
 'question': 122,
 'detroit': 5,
 'another': 301,
 'dim': 44,
 'intelligence': 44,
 'subconscious': 6,
 'simmons': 1,
 'wait': 175,
 'antartica': 1,
 'francisco': 17,
 'cleaned': 34,
 'ankle': 16,
 'protruded': 4,
 'transparent': 16,
 'people': 270,
 'necessities': 14,
 'literally': 30,
 'larger': 67,
 'evil': 104,
 'lifestyle': 9,
 'emotional': 19,
 'felt': 213,
 'time': 395,
 'good': 296,
 'plans': 69,
 'plastic': 51,
 'trying': 181,
 'populous': 4,
 'well': 332,
 'small': 222,
 'said': 348,
 'six': 262,
 'struc': 1,
 'saw': 262,
 'buckets': 4,
 'occasionally': 47,
 'death': 164,
 'identification': 11,
 'holographic': 6,
 'age': 115,
 'ancestors': 13,
 'dock': 21,
 'give': 230,
 'likely': 78,
 'closed': 118,
 'person':

In [134]:
print(len(doc_frequency))
N = len(paths)
print('Total Number of documents are',N)

49885
Total Number of documents are 467


***Inverse Document Frequency***

In [135]:
#Inverse Document Frequency IDF(word)=log(total no. of documents/document frequency(word)+1)
inverse_document_frequency = {}
keys_df = doc_frequency.keys()
for item in keys_df:
    df = doc_frequency[item]   #document frequency of each word in document frquency is picked
    inv_doc_freq = np.log(N/df+1)
    inverse_document_frequency[item] = inv_doc_freq

In [136]:
inverse_document_frequency

{'found': 0.9948725189080168,
 'atmospheric': 4.547541073151455,
 'offer': 2.013049451892942,
 'door': 1.0722035266948127,
 'years': 1.100041881477704,
 'isaak': 6.148468295917647,
 'liquid': 2.539631549477247,
 'physical': 2.250334402650168,
 'outside': 1.384690511537916,
 'toward': 1.5551928975817046,
 'white': 1.2714260296272948,
 'possibly': 1.861010705268827,
 'zero': 1.0307219531252585,
 'st': 2.372965889755845,
 'sworn': 3.018390297446575,
 'cried': 1.6215383193681878,
 'mouth': 1.351969618578816,
 'match': 2.1728559863192616,
 'question': 1.57440513891833,
 'detroit': 4.547541073151455,
 'another': 0.9366794683987963,
 'dim': 2.4521799562854434,
 'intelligence': 2.4521799562854434,
 'subconscious': 4.367335919263878,
 'simmons': 6.148468295917647,
 'wait': 1.2998023297664472,
 'antartica': 6.148468295917647,
 'francisco': 3.3488715626604155,
 'cleaned': 2.6902455764687034,
 'ankle': 3.4074279314127915,
 'protruded': 4.768563732896527,
 'transparent': 3.4074279314127915,
 'peopl

***Term frequency–Inverse document frequency***


In [137]:
#Function block for Calculating tf-idf value---tf*idf as tf=idf will be calculated based on all the 5 variants
def calculate_tfidf(list_tf):
    final_tfidf_list = []  #creating list of list of tf-idf
    for item in list_tf:
        dict_tfidf = {}
        allkeys = item.keys()
        for w in allkeys:
            term_freq = item[w]   #tf
            idf_for_word = inverse_document_frequency[w]  #idf
            tfidf = term_freq * idf_for_word              #Multiplying tfidf=tf*idf
            dict_tfidf[w] = tfidf
        final_tfidf_list.append(dict_tfidf)
    return final_tfidf_list

In [138]:
#calculating tfidf for all 5 variants
tfidf_binary = calculate_tfidf(Binary_Term_Frequency)
tfidf_raw_count = calculate_tfidf(Raw_Count_Frequency)
tfidf_term_frequency = calculate_tfidf(Term_Frequency)
tfidf_log_normalisation = calculate_tfidf(Log_Normalisation)
tfidf_double_normalisation = calculate_tfidf(Double_Normalisation)

In [139]:
print(tfidf_term_frequency[1])

{'volume': 0.0026114802516351903, 'one': 0.009170168791157713, 'number': 0.004870232149958529, 'two': 0.004387300784018728, 'author': 0.0026568460045443606, 'caroline': 0.010156685267085255, 'kent': 0.008451955861331632, 'mail': 0.002568300797318078, 'caro': 0.005811986781561591, 'freenet': 0.005382449846683701, 'fsu': 0.005811986781561591, 'edu': 0.003766570756645903, 'copyright': 0.0017729071123181552, 'nine': 0.003930833850393067, 'five': 0.0010474055573827426, 'rights': 0.002143822632473847, 'reserved': 0.002414021294131089, 'minutes': 0.0029699222140997864, 'zero': 0.0021953609225245124, 'vigorously': 0.00362878373952374, 'wiping': 0.00362878373952374, 'front': 0.008901493219811046, 'door': 0.02397899260978814, 'bookstore': 0.029057770435472557, 'trying': 0.002716493429364109, 'remove': 0.0025072273347933297, 'stubborn': 0.0041160078784816785, 'spot': 0.0017970677557003966, 'hi': 0.00243188168053605, 'welcome': 0.0020473079528733756, 'back': 0.004513876146839955, 'book': 0.0172687

**Creating tf-idf matrix for all 5 variants**

In [140]:
len(tfidf_raw_count)
all_terms = list(doc_frequency.keys())
len(all_terms)
all_terms

['found',
 'atmospheric',
 'offer',
 'door',
 'years',
 'isaak',
 'liquid',
 'physical',
 'outside',
 'toward',
 'white',
 'possibly',
 'zero',
 'st',
 'sworn',
 'cried',
 'mouth',
 'match',
 'question',
 'detroit',
 'another',
 'dim',
 'intelligence',
 'subconscious',
 'simmons',
 'wait',
 'antartica',
 'francisco',
 'cleaned',
 'ankle',
 'protruded',
 'transparent',
 'people',
 'necessities',
 'literally',
 'larger',
 'evil',
 'lifestyle',
 'emotional',
 'felt',
 'time',
 'good',
 'plans',
 'plastic',
 'trying',
 'populous',
 'well',
 'small',
 'said',
 'six',
 'struc',
 'saw',
 'buckets',
 'occasionally',
 'death',
 'identification',
 'holographic',
 'age',
 'ancestors',
 'dock',
 'give',
 'likely',
 'closed',
 'person',
 'view',
 'kissed',
 'curiosity',
 'tures',
 'giggled',
 'walked',
 'plagued',
 'ignored',
 'inaccuracy',
 'new',
 'domingo',
 'stops',
 'soil',
 'looked',
 'waited',
 'scanning',
 'program',
 'replied',
 'sighted',
 'understood',
 'physically',
 'manners',
 'eyes',

In [141]:
#tf-idf score using log
tfidf_log_matrix = []  #2-D matrix of doc * vocab size storing tfidf values of log variant
for item in tfidf_log_normalisation:
    doc_log = []
    for w in all_terms:
        if w in item :
            temp = item[w]
            doc_log.append(temp)    #if word present then store tfidf
        else:
            doc_log.append(0.0)    #else 0
    tfidf_log_matrix += [doc_log]   

In [142]:
#tf-idf score using binary
tfidf_binary_matrix = []   #2-D matrix of doc * vocab size storing tfidf values of binary variant
for item in tfidf_binary:
    doc_bin = []
    for w in all_terms:
        if w in item :
            temp = item[w]
            doc_bin.append(temp)   #if word present then store tfidf
        else:
            doc_bin.append(0.0)    #else 0
    tfidf_binary_matrix += [doc_bin]   

In [143]:
#tf-idf score using double
tfidf_double_matrix = []    #2-D matrix of doc * vocab size storing tfidf values of double normalisation variant
for item in tfidf_double_normalisation:
    doc_db = []
    for w in all_terms:
        if w in item :
            temp = item[w]
            doc_db.append(temp)    #if word present then store tfidf
        else:
            doc_db.append(0.0)    #else 0
    tfidf_double_matrix += [doc_db]   

In [144]:
##tf-idf score using term frequency
tfidf_term_matrix = []   #2-D matrix of doc * vocab size storing tfidf values of term frequency variant
for item in tfidf_term_frequency:
    doc_term = []
    for w in all_terms:
        if w in item :
            temp = item[w]
            doc_term.append(temp)    #if word present then store tfidf
        else:
            doc_term.append(0.0)    #else 0
    tfidf_term_matrix += [doc_term]   

In [145]:
#tf-idf score using raw count
tfidf_raw_matrix = []   #2-D matrix of doc * vocab size storing tfidf values of raw count frequency variant
for item in tfidf_raw_count:
    doc_inside = []
    for word in all_terms:
        if word in item :
            temp = item[word]
            doc_inside.append(temp)   #if word present then store tfidf
        else:
            doc_inside.append(0.0)     #else 0
    tfidf_raw_matrix += [doc_inside]        

In [146]:
#fetching rank document based on tfidf
def tfidf_rank_doc(tfidf_variant_list,query_list):
    #creating query vector of vaocab size  
    query_vector = [0.0] * len(all_terms)
    #print(tfidf_variant_list)
    #print(query_list)
    temp = set(query_list)
    res = [i for i, val in enumerate(all_terms) if val in temp]
    print(res)
    for i in range(len(all_terms)):
      if i in res:
        query_vector[i]=1.0         #for query words which are present in vocab value assigned as 1
        print('query vector ',query_vector[i])
      else:
        query_vector[i]=0.0
    print('query vector ',query_vector)

    doc_rank = {}
    #tf_idf_query_vector=list()
    for i in range(N):
        doc_rank[i] = 0.0
    for query_word in res:
        #print('dbjbdjd',query_word)
        count = -1
        for item in tfidf_variant_list:
            count += 1
            score = item[query_word]
            doc_rank[count] += score
        #print("score",score)
    doc_rank = dict(sorted(doc_rank.items(),key=operator.itemgetter(1),reverse=True))   
    print(doc_rank)
    return doc_rank,query_vector

In [147]:
def top5_tfidf_doc(list_tfidf,query_list,k,variant):
    #print(list_tfidf)
    list_tf ,query_vector= tfidf_rank_doc(list_tfidf,query_list)
    #print(list_tf)
    #print("query_vctor",query_vector)
    count = -1
    flag = 0
    list_final = []
    for item in list_tf.keys():
        count += 1
        if count == k:
            print('Top ',k,' Documents based on ',variant,' tf-idf are',list_final)
            for i in list_final:
                print(fileIdName.iloc[i][0])
            flag = 1
            break
        list_final.append(item)
    if flag == 0:
        print('Top ',k,' Documents based on ',variant,' tf-idf are:',list_final)    
        for i in list_final:
            print(fileIdName.iloc[i][0])
    return query_vector         

In [148]:
query = input('Enter the String   : ')
#fetching all query vector
query_afterPreprocess =query_tokens(query)  # fetching query tokens
#fetching tfidf top 5 documents based on all 5 variants
query_vector_raw=top5_tfidf_doc(tfidf_raw_matrix,query_afterPreprocess,5,'Raw_Count')
query_vector_log=top5_tfidf_doc(tfidf_log_matrix,query_afterPreprocess,5,'Logarithmic')   
query_vector_term=top5_tfidf_doc(tfidf_term_matrix,query_afterPreprocess,5,'termfrequency')
query_vector_binary=top5_tfidf_doc(tfidf_binary_matrix,query_afterPreprocess,5,'Binary')
query_vector_double=top5_tfidf_doc(tfidf_double_matrix,query_afterPreprocess,5,'Double') 

Enter the String   : good day
[41, 111]
query vector  1.0
query vector  1.0
query vector  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

**2.3-----Cosine Similarity**

In [149]:
#converting into np array
query_vector_log=np.array(query_vector_log)
query_vector_raw=np.array(query_vector_raw)
query_vector_binary=np.array(query_vector_binary)
query_vector_term=np.array(query_vector_term)
query_vector_double=np.array(query_vector_double)

In [150]:
#reshaping
query_vector_log=query_vector_log.reshape(1,49885)
query_vector_raw=query_vector_log.reshape(1,49885)
query_vector_binary=query_vector_log.reshape(1,49885)
query_vector_term=query_vector_log.reshape(1,49885)
query_vector_double=query_vector_log.reshape(1,49885)

In [151]:
print(query_vector_log.shape)
print(query_vector_raw.shape)

(1, 49885)
(1, 49885)


In [152]:
#converting into list
query_vector_log=query_vector_log.tolist()
query_vector_raw=query_vector_raw.tolist()
query_vector_binary=query_vector_binary.tolist()
query_vector_term=query_vector_term.tolist()
query_vector_double=query_vector_double.tolist()

In [153]:
query_vector_raw[0][147]

0.0

In [154]:
#cosine similarity using wtd. score
def find_wtd_query_vector_raw(query):
  
    text_of_query = word_tokenize(str(preprocess(query)))
    
    #for frequency of term in query
    query_term_freq= {}

    for item in text_of_query:
    
        if item in query_term_freq:
            temp = query_term_freq[item]
            temp += 1
            query_term_freq[item] = temp
        else:
             query_term_freq[item] = 1
    
    
    # Coverting term frequency into raw frequency

    for item in query_term_freq.keys():
    
        temp = query_term_freq[item]
        new = temp
        query_term_freq[item] = new
    
    #This is for storing inverse_document_frequency
    tf_idf_query = {}

    for item in query_term_freq.keys():
    
        if item in inverse_document_frequency :
        
            temp = inverse_document_frequency[item]
            toadd = temp * query_term_freq[item]
        
            tf_idf_query[item] = toadd
        
        else:
            tf_idf_query[item] = 0.0
    
    
    #  query vector 

    query_vector = []

    for word in all_terms :
    
        if word in tf_idf_query :
            temp = tf_idf_query[word]
            query_vector.append(temp)
        else:
            query_vector.append(0.0)
    return query_vector        

In [155]:
#finding weighted query vector using binary
def find_wtd_query_vector_binary(query):
  
    text_of_query = word_tokenize(str(preprocess(query)))
    
    #term_frequncy of query
    query_term_freq = {}\

    for item in text_of_query:
    
        if item in query_term_freq:
            temp = query_term_freq[item]
            temp += 1
            query_term_freq[item] = temp
        else:
             query_term_freq[item] = 1
    
    
    # Coverting into log frequency

    for item in query_term_freq.keys():
    
        temp = query_term_freq[item]
        if temp > 0:
            new = 1
        else:
            new = 0
        query_term_freq[item] = new
    
    #inverse_document_frequency
    tf_idf_query = {}

    for item in query_term_freq.keys():
    
        if item in inverse_document_frequency :
        
            temp = inverse_document_frequency[item]
            toadd = temp * query_term_freq[item]
        
            tf_idf_query[item] = toadd
        
        else:
            tf_idf_query[item] = 0.0
    
    
    #  query vector 

    query_vector = []

    for word in all_terms :
    
        if word in tf_idf_query :
            temp = tf_idf_query[word]
            query_vector.append(temp)
        else:
            query_vector.append(0.0)
    return query_vector        

In [156]:
#finding weighted query vector using term
def find_wtd_query_vector_term(query):
    #for tokenization of query
    text_of_query = word_tokenize(str(preprocess(query)))
    
    #finding term frequency of the query
    query_term_freq = {}
    
    for item in text_of_query:
    
        if item in query_term_freq:
            temp = query_term_freq[item]
            temp += 1
            query_term_freq[item] = temp
        else:
             query_term_freq[item] = 1
    
    sum_tf=sum(query_term_freq.values())
    # Coverting into term frequency

    for item in query_term_freq.keys():
    
        temp = query_term_freq[item]
        new = (temp/sum_tf)
        query_term_freq[item] = new
    
    #inverse_document_frequency
    tf_idf_query = {}

    for item in query_term_freq.keys():
    
        if item in inverse_document_frequency :
        
            temp = inverse_document_frequency[item]
            toadd = temp * query_term_freq[item]
        
            tf_idf_query[item] = toadd
        
        else:
            tf_idf_query[item] = 0.0
    
    
    #  query vector 

    query_vector = []

    for word in all_terms :
    
        if word in tf_idf_query :
            temp = tf_idf_query[word]
            query_vector.append(temp)
        else:
            query_vector.append(0.0)
    return query_vector        

In [157]:
##finding weighted query vector using log
def find_wtd_query_vector_log(query):
   #preproceesing the query
    text_of_query = word_tokenize(str(preprocess(query)))
    
    #query_term_freq
    query_term_freq = {}

    for item in text_of_query:
    
        if item in query_term_freq:
            temp = query_term_freq[item]
            temp += 1
            query_term_freq[item] = temp
        else:
             query_term_freq[item] = 1
    
    
    # Coverting into log frequency

    for item in query_term_freq.keys():
    
        temp = query_term_freq[item]
        new = np.log(1+temp)
        query_term_freq[item] = new
    
    #inverse_document_frequency
    tf_idf_query = {}

    for item in query_term_freq.keys():
    
        if item in inverse_document_frequency :
        
            temp = inverse_document_frequency[item]
            toadd = temp * query_term_freq[item]
        
            tf_idf_query[item] = toadd
        
        else:
            tf_idf_query[item] = 0.0
    
    
    #  query vector 

    query_vector = []

    for word in all_terms :
    
        if word in tf_idf_query :
            temp = tf_idf_query[word]
            query_vector.append(temp)
        else:
            query_vector.append(0.0)
    return query_vector        

In [158]:
##finding weighted query vector using double
def find_wtd_query_vector_double_norm(query):
   #preprocessing the query
    text_of_query = word_tokenize(str(preprocess(query)))
    
    #query_term_freq
    query_term_freq = {}

    for item in text_of_query:
    
        if item in query_term_freq:
            temp = query_term_freq[item]
            temp += 1
            query_term_freq[item] = temp
        else:
             query_term_freq[item] = 1
    
    
    # Coverting into double frequency
    max_tf = max(query_term_freq.values())
    for item in query_term_freq.keys():
        temp = query_term_freq[item]
        new =  0.5 + (0.5*(temp/max_tf))
        query_term_freq[item] = new
    
    #inverse_document_frequency
    tf_idf_query = {}

    for item in query_term_freq.keys():
    
        if item in inverse_document_frequency :
        
            temp = inverse_document_frequency[item]
            toadd = temp * query_term_freq[item]
        
            tf_idf_query[item] = toadd
        
        else:
            tf_idf_query[item] = 0.0
    
    
    #  query vector 

    query_vector = []

    for word in all_terms :
    
        if word in tf_idf_query :
            temp = tf_idf_query[word]
            query_vector.append(temp)
        else:
            query_vector.append(0.0)
    return query_vector        

In [159]:
#cosine similarity function weighted
def cosinevalues_wtd(query,doc_vector,tfidfqv):
    
    #This list will store normalized value of doc vectors
    
    doc_vec_normalized = []

    for item in doc_vector:
        #for every doc list
    
        nom= 0.0
        for freq in item:
            nom+= (freq*freq)
    
        nom= 1 / np.sqrt(nom)
    
        temp_list = []
    
        for freq in item:
            k = freq * nom
            temp_list.append(k)
        
        doc_vec_normalized.append(temp_list)
        
    #This list will store Normalized value of query vector    
    
    query_vec_normalized = []

    normal = 0.0
    for item in tfidfqv:
        #print("itjinvn",item)
        normal += (item*item)
    
    normal = 1/np.sqrt(normal)
    for item in tfidfqv:
        query_vec_normalized.append(item * normal)
    
    
    #Finding Cosine Scores!!!
    
    cosine_scores = {}

    documentId = -1
    for item in doc_vec_normalized:
    
        documentId += 1
        list_doc = np.array(item)
        list_query = np.array(query_vec_normalized)
        score = sum(list(list_doc * list_query))
    
        cosine_scores[documentId] = score
    
    sorted_cosine_scores = dict(sorted(cosine_scores.items(),key=operator.itemgetter(1),reverse=True))
    
 
    
    
    
    return sorted_cosine_scores

In [160]:
#normal cosine similarity function
def cosinevalues(query,doc_vector,tfidfqv):
   
    #This vector will store Normalized value  of doc vectors
    
    doc_vec_normalized = []

    for item in doc_vector:
        #each doc list
    
        nom= 0.0
        for freq in item:
            nom+= (freq*freq)
    
        nom= 1 / np.sqrt(norm)
    
        temp_list = []
    
        for freq in item:
            k = freq * norm
            temp_list.append(k)
        
        doc_vec_normalized.append(temp_list)
        
    #This vector will store Normalized value of query vector    
    
    query_vec_normalized = []

    normal = 0.0
    for item in tfidfqv[0]:
        
        normal += (item*item)
    
    normal = 1/np.sqrt(normal)
    for item in tfidfqv[0]:
        query_vec_normalized.append(item * normal)
    
    
    #Finding Cosine Scores!!!
    
    cosine_scores = {}

    documentId = -1
    for item in doc_vec_normalized:
    
        documentId += 1
        list_doc = np.array(item)
        list_query = np.array(query_vec_normalized)
        score = sum(list(list_doc * list_query))
    
        cosine_scores[documentId] = score
    
    sorted_cosine_scores = dict(sorted(cosine_scores.items(),key=operator.itemgetter(1),reverse=True))
    

    
    
    
    return sorted_cosine_scores

In [161]:
#calling cosine fucntion for different approach and printing top 5 documents
def execute_three(getInputthree):
    k=5
    cosine_value1 = cosinevalues(getInputthree,tfidf_log_matrix,query_vector_log)
    cosine_value2 = cosinevalues(getInputthree,tfidf_raw_matrix,query_vector_raw)
    cosine_value3 = cosinevalues(getInputthree,tfidf_binary_matrix,query_vector_binary)
    cosine_value4 = cosinevalues(getInputthree,tfidf_double_matrix,query_vector_double)
    cosine_value5 = cosinevalues(getInputthree,tfidf_term_matrix,query_vector_term)
    string = ""
    count = -1
    flag = 0
    list_final = []
    for item in cosine_value1.keys():
        count += 1
        if count == k:
            print('Top',k,' Documents found based on',string,' tf-idf score using log method are',list_final)
            for i in list_final:
                print(fileIdName.iloc[i])
#                 print(titles[i]+ " - "+ file_name[i])
            flag = 1
            break
        list_final.append(item)
    string = ""
    count = -1
    flag = 0
    list_final = []
    for item in cosine_value2.keys():
        count += 1
        if count == k:
            print('Top ',k,' Documents  found on basis of ',string,' tf-idf score are using raw method',list_final)
            for i in list_final:
                print(fileIdName.iloc[i])
#                 print(titles[i]+ " - "+ file_name[i])
            flag = 1
            break
        list_final.append(item)
    string = ""
    count = -1
    flag = 0
    list_final = []
    for item in cosine_value3.keys():
        count += 1
        if count == k:
            print('Top ',k,' Documents found on basis of ',string,' tf-idf score are using binary methid',list_final)
            for i in list_final:
                print(fileIdName.iloc[i])
#                 print(titles[i]+ " - "+ file_name[i])
            flag = 1
            break
        list_final.append(item)
    string = ""
    count = -1
    flag = 0
    list_final = []
    for item in cosine_value4.keys():
        count += 1
        if count == k:
            print('Top ',k,' Documents found on basis of ',string,' tf-idf score are using double method',list_final)
            for i in list_final:
                print(fileIdName.iloc[i])
#                 print(titles[i]+ " - "+ file_name[i])
            flag = 1
            break
        list_final.append(item)
    string = ""
    count = -1
    flag = 0
    list_final = []
    for item in cosine_value5.keys():
        count += 1
        if count == k:
            print('Top ',k,' Documents found on basis of ',string,' tf-idf scoreare using term method',list_final)
            for i in list_final:
                print(fileIdName.iloc[i])
#                 print(titles[i]+ " - "+ file_name[i])
            flag = 1
            break
        list_final.append(item)

In [162]:
#calling cosine similarity function wtd one and printing top 5 documents
def execute_three_wtd(getInputthree):
    k=5
    query_vector_log=find_wtd_query_vector_log(getInputthree)
    query_vector_raw=find_wtd_query_vector_raw(getInputthree)
    query_vector_binary=find_wtd_query_vector_binary(getInputthree)
    query_vector_double=find_wtd_query_vector_double_norm(getInputthree)
    query_vector_term=find_wtd_query_vector_term(getInputthree)
    cosine_value1 = cosinevalues_wtd(getInputthree,tfidf_log_matrix,query_vector_log)
    cosine_value2 = cosinevalues_wtd(getInputthree,tfidf_raw_matrix,query_vector_raw)
    cosine_value3 = cosinevalues_wtd(getInputthree,tfidf_binary_matrix,query_vector_binary)
    cosine_value4 = cosinevalues_wtd(getInputthree,tfidf_double_matrix,query_vector_double)
    cosine_value5 = cosinevalues_wtd(getInputthree,tfidf_term_matrix,query_vector_term)
    string = ""
    count = -1
    flag = 0
    list_final = []
   
    for item in cosine_value1.keys():
        count += 1
        if count == k:
            print('Top ',k,' Documents found on basis of ',string,' tf-idf using log method are',list_final)
            for i in list_final:
                print(fileIdName.iloc[i])
#                 print(titles[i]+ " - "+ file_name[i])
            flag = 1
            break
        list_final.append(item)
    string = ""
    count = -1
    flag = 0
    list_final = []
    for item in cosine_value2.keys():
        count += 1
        if count == k:
            print('Top ',k,' Documents found on basis of ',string,' tf-idf using raw method are',list_final)
            for i in list_final:
                print(fileIdName.iloc[i])
#                 print(titles[i]+ " - "+ file_name[i])
            flag = 1
            break
        list_final.append(item)
    string = ""
    count = -1
    flag = 0
    list_final = []
    for item in cosine_value3.keys():
        count += 1
        if count == k:
            print('Top ',k,' Documents found on basis of ',string,' tf-idf using binary methodare',list_final)
            for i in list_final:
                print(fileIdName.iloc[i])
#                 print(titles[i]+ " - "+ file_name[i])
            flag = 1
            break
        list_final.append(item)
    string = ""
    count = -1
    flag = 0
    list_final = []
    for item in cosine_value4.keys():
        count += 1
        if count == k:
            print('Top ',k,' Documents found on basis of',string,' tf-idf using double method are',list_final)
            for i in list_final:
                print(fileIdName.iloc[i])
#                 print(titles[i]+ " - "+ file_name[i])
            flag = 1
            break
        list_final.append(item)
    string = ""
    count = -1
    flag = 0
    list_final = []
    for item in cosine_value5.keys():
        count += 1
        if count == k:
            print('Top ',k,' Documents found on basis of ',string,' tf-idf using term are',list_final)
            for i in list_final:
                print(fileIdName.iloc[i])
#                 print(titles[i]+ " - "+ file_name[i])
            flag = 1
            break
        list_final.append(item)

In [163]:
#output of cosine similarity
getInputthree = input('Enter the String   : ')

print()
#execute_three(getInputthree)
execute_three_wtd(getInputthree)

Enter the String   : good day

Top  5  Documents found on basis of    tf-idf using log method are [5, 98, 150, 49, 88]
0    bestwish
Name: 5, dtype: object
0    horswolf.txt
Name: 98, dtype: object
0    mouslion.txt
Name: 150, dtype: object
0    pepdegener.txt
Name: 49, dtype: object
0    pepsi.degenerat
Name: 88, dtype: object
Top  5  Documents found on basis of    tf-idf using raw method are [269, 49, 88, 229, 442]
0    blossom.pom
Name: 269, dtype: object
0    pepdegener.txt
Name: 49, dtype: object
0    pepsi.degenerat
Name: 88, dtype: object
0    brain.damage
Name: 229, dtype: object
0    contrad1.hum
Name: 442, dtype: object
Top  5  Documents found on basis of    tf-idf using binary methodare [5, 98, 416, 150, 353]
0    bestwish
Name: 5, dtype: object
0    horswolf.txt
Name: 98, dtype: object
0    elveshoe.txt
Name: 416, dtype: object
0    mouslion.txt
Name: 150, dtype: object
0    aminegg.txt
Name: 353, dtype: object
Top  5  Documents found on basis of   tf-idf using double metho